In [6]:
import pandas as pd
import numpy as np

There is a nice elementary proof of this series using the [Fibonacci generating function][1]. Consider the sum
$$
\begin{align*}
(1 - x - x^2)\sum_{n=1}^{\infty} F_{n}x^{n} &= \sum_{n=1}^{\infty} F_{n}x^{n} - \sum_{n=1}^{\infty} F_{n}x^{n+1} - \sum_{n=1}^{\infty} F_{n}x^{n+2} \\
&= \sum_{n=1}^{\infty} F_{n}x^{n} - \sum_{n=2}^{\infty} F_{n-1}x^{n} - \sum_{n=3}^{\infty} F_{n-2}x^{n} \\
&= \left[F_1 x + F_2 x^2 + \sum_{n=3}^{\infty} F_{n}x^{n} \right] - \left[F_{1}x^2 + \sum_{n=3}^{\infty} F_{n-1}x^{n}\right] - \sum_{n=3}^{\infty} F_{n-2}x^{n} \\
&= F_1 x + (F_2 - F_1) x^2 + \sum_{n=1}^{\infty} (F_{n} - F_{n-1} - F_{n-2})x^{n} \\
&= x 
\end{align*}
$$

The last line uses the definition of the Fibonacci sequece: $F_1 = F_2 = 1$ and that $F_{n} = F_{n-1} + F_{n-2}$ to show the second term and the summation are both $0$. So we get
$$
(1 - x - x^2)\sum_{n=1}^{\infty} F_{n}x^{n} = x \implies \sum_{n=1}^{\infty} F_{n}x^{n} = \frac{x}{1 - x - x^2}.
$$

If this must be an integer $k$, we get
$$
x = k(1 - x - x^2) \implies kx^2 + (k+1)x - k = 0 \implies x = \frac{-(k+1) \pm \sqrt{(k+1)^2 + 4k^2}}{2}
$$
which can only have a rational solution if the discriminant is an integer. So we need to check each $k$ such that $(k+1)^2 + (2k)^2$ is a perfect square. 

A couple ways to describe the problem with some skills we did in previous problems:
1. This is basically generating a Pythagorean triple $(a,b,c)$ with the requirement that $a-1 = b/2$.
2. This is solving the Diophantine equation $x^2 + 4y^2 = z^2$ with the requirement $x = y+1$.

However, these are too slow. It turns out this is a [known series][1] and that any rational $x^*$ has an associated $n \geq 0$ such that $A_F(x^*) = F_{2n} \cdot F_{2n+1}$ is an integer, with $F_0 = 0$ and $F_1 = 1$. The proof is quite complicated from what I saw.

[1]: https://en.wikipedia.org/wiki/Fibonacci_sequence#Generating_function
[2]: https://oeis.org/A081018

In [12]:
def valid_ints(number_needed):
    fib = [0, 1]
    for i in range(2*number_needed + 1):
        fib.append(fib[-1] + fib[-2])

    return [fib[2*i]*fib[2*i + 1] for i in range(number_needed+1)]

In [13]:
valid_ints(number_needed=15)[15]

1120149658760